# Исследование объявлений о продаже квартир

### Откройте файл с данными и изучите общую информацию. 

In [ ]:
# импортируем необходимые библиотеки
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%pip install pymystem3==0.1.10
from pymystem3 import Mystem
m = Mystem()

from IPython.display import display
pd.set_option('display.float_format', '{:,.2f}'.format)

pd.options.mode.chained_assignment = None  # Удалим предупреждения

In [ ]:
# прочитаем файл с данными и сохраним в таблицу df, выставим разделитель sep='\t'
try:
    df = pd.read_csv('/datasets/real_estate_data.csv', sep='\t')
except:
    df = pd.read_csv(r"C:\Users\User\OneDrive\Education\1_Ya_DA\7_research_data_analysis\project_3\real_estate_data.csv", sep='\t')


In [ ]:
# выведем первые 10 строк датафрейма df на экран
df.head(10)

In [ ]:
# Получим общую информацию о данных в таблице df
df.info()

In [ ]:
# Получим описательную статистику данных таблицы df
df.describe()

**Вывод**

В таблице 23699 строк и 22 колонки. Типы данных: `int64`, `float64`, `object`, `bool`.  

Количество значений в колонках различается, а значит, в данных есть пропущенные значения.

### Предобработка данных

In [ ]:
# Выведем на экран названия столбцов таблицы df
df.columns

In [ ]:
# Представим названия столбцов в соответствии с хорошим стилем
df = df.rename(columns={'cityCenters_nearest': 'city_centers_nearest',
                       'parks_around3000': 'parks_around_3000',
                       'ponds_around3000': 'ponds_around_3000'})

In [ ]:
# Проверим таблицу df на количество дубликатов
print('Количество дубликатов =', df.duplicated().sum())

In [ ]:
# Выведем количество пропущенных значений для каждого столбца
df.isna().sum().sort_values(ascending=False)

In [ ]:
# Посмотрим процент пропусков в столбцах с сортировкой по убыванию
(df.isna().mean().sort_values(ascending=False) * 100).round()

В 12-и колонках из 22-ух пропуски данных

In [ ]:
# Изменим тип данных в столбце first_day_exposition - дата публикации, 
# на datetime
df['first_day_exposition'] = pd.to_datetime(df['first_day_exposition'], format='%Y-%m-%d')

In [ ]:
# Получим описательную статистику колонки last_price - 
# цена на момент снятия с публикации
df['last_price'].describe()

Проверим минимальное значение - 12,190.00, в сравнении с другими объектами с минимальной стоимостью.

In [ ]:
# Отфильтруем данные применив метод query
df.query('last_price < 450_000')

Сомнительная цена для 2-ух комнатной квартиры общей площадью 109 м2. Учитывая, что средняя цена - 6,541,548.77, а медиана - 4,650,000.00, предположительно цена указана в тысячах рублей и должна быть - 12,190,000.00

In [ ]:
# Используем индексацию c умножением для изменения значения цены
df.loc[8793, 'last_price'] = df.loc[8793, 'last_price'] * 1000

Проверим максимальное значение - 763,000,000.00, в сравнении с другими объектами с высокой стоимостью

In [ ]:
# Отфильтруем данные применив метод query
df.query('last_price > 350_000_000')

Объект сильно выбивается из ценовой категории в сравнении с ближайшими "кокурентами", цена сомнительна. Однако, можно предположить, что это пентхаус, так как квартира находится на 10-ом этаже 10-и этажного дома.

**Вывод:**

При рассмотрении значений в колонке `last_price` были обнаружены сомнительные значения минимальной и максимальной цены. Значение минимальной цены было исправлено как ошибка заполнения, максимальная цена, видимо реальна.

---

In [ ]:
# Получим описательную статистику колонки ceiling_height
df['ceiling_height'].describe()

Ранее, методом df.isna().sum(), было обнаружено 9195 пропусков. Кроме этого, в колонке так же есть явные отклонения:
- минимальное значение высоты потолка - 1 м,
- максимальное значение высоты потолка - 100 м.
Всё это маловероятно.

Заменим значения меньше 2 м на медиану

In [ ]:
# Используем индексацию с подстановкой медианных значений
df.loc[(df['ceiling_height'] < 2), 'ceiling_height'] = df['ceiling_height'].median()

В дорогих квартирах (например: 'https://realty.yandex.ru/offer/5535692837356630853/') высота потолка достигает 5-и метров, можно предположить, что есть квартиры с высотой потолка 6 метров, либо 2-ух уровневые квартиры с потолком бОльших размеров.

Посмотрим на значения где потолок выше 7-и метров

In [ ]:
# # Отфильтруем данные применив метод query
df.query('ceiling_height > 7')



Видим двузначные значения. Как мы предполагали ранее, данная аномалия связана с неправильным указанием в объявлениях размеров потолка. Разделим значения больше 20 метров на 10, то есть сдвинем разделитель на один знак влево, а пропускам и значениям больше 7 метров присвоим медиану.

In [ ]:
# Используем индексацию с делением на 10
df.loc[(df['ceiling_height'] > 20), 'ceiling_height'] = df['ceiling_height'] / 10

# Используем индексацию с подстановкой медианных значений
df.loc[(df['ceiling_height'] > 7), 'ceiling_height'] = df['ceiling_height'].median()

# Используя метод fillna заполним недостающие пропуски (9,195) медианным значением 
df['ceiling_height'] = df['ceiling_height'].fillna(df['ceiling_height'].median())

In [ ]:
# Проверим количество пропущенных значений в ceiling_height
print('Количество отсутствующих значений -', df['ceiling_height'].isna().sum())

В колонке `floors_total` - всего этажей в доме, 86 пропусков. Мы не знаем, какое количество этажей в доме, тогда предположим, что данные объекты находятся на последних этажах.

In [ ]:
# Удалим пропуски
df = df.dropna(subset=['floors_total'])

In [ ]:
# Проверим количество пропущенных значений  в floors_total
print('Количество отсутствующих значений -', df['floors_total'].isna().sum())

В колонке `living_area` — жилая площадь в квадратных метрах (м²), 1,903 пропуска.

Предположим, что доля жилой площади может зависить как от общей площади, так и от количества комнат в помещении. Для заполнения пропусков разобьем значения на группы по количеству комнат и найдем долю жилой площади в размере общей площади.

Создадим таблицу с долей жилой площади в общей площади в зависимости от количества комнат

In [ ]:
# Применим группировку в разрезе комната к жилой и общей площади
living_area_data = (pd.DataFrame(df.groupby('rooms')['living_area'].median()
                                /df.groupby('rooms')['total_area'].median()))

# добавим колонку living_area_data в таблицу df
living_area_data.columns = ['living_area_data']

# выведем первые 5 строк
living_area_data.head()

Объединим две таблицы с заполнением пропусков произведением общей площади на долю жилой площади с округлием значения. Удалим колонку living_area_data.

In [ ]:
# присоединим к таблице df, таблицу living_area_data, по общему 
# столбцу rooms с аргументом left (таблица df)
df = df.merge(living_area_data, on='rooms', how='left')

# Вставим в пропущенные значения произведение общей площади на долю жилой площади
df['living_area'] = df['living_area'].fillna(df['total_area'] * df['living_area_data'])

# Удалим колонку living_area_data
df = df.drop(columns = ['living_area_data'], axis=1)

# Выведем первые 5 строк
df.head()

In [ ]:
# Проверим количество пропущенных значений в колонке living_area
print('Количество отсутствующих значений -', df['living_area'].isna().sum())

В колонке `is_apartment` — апартаменты(булев тип), 20,924 пропущенных значений. Предположим, что формат аппартаментов мало распостранен и пропущенные значения указывают на то, что объекты не являются аппартаментами.

Заполним пропуски в колонке is_apartment значением - False.

In [ ]:
# Заполним пропущенные значения булевым типом - False, методом fillna
df['is_apartment'] = df['is_apartment'].fillna(False)

In [ ]:
# Проверим количество пропущенных значений в колонке is_apartment
print('Количество отсутствующих значений -', df['is_apartment'].isna().sum())

В колонке `kitchen_area` — площадь кухни в квадратных метрах (м²), 2,278 пропущенных значений. Допустим наличие зависимости между количеством комнат и площадью кухни. Заполним пропуски медианным значением в группах в зависимости от количества комнат. 

In [ ]:
# Заполним пропущенные значения методом fillna c группировкой по столбцу rooms 
# c указанием на данные колонки kitchen_area c применением метода transform
df['kitchen_area'] = df['kitchen_area'].fillna(df.groupby('rooms')['kitchen_area'].transform(func='median'))

In [ ]:
# Проверим количество пропущенных значений в колонке kitchen_area
print('Количество отсутствующих значений -', df['kitchen_area'].isna().sum())

Остались пропущенные значения. Предположительно, что это студии, в которых комната и кухня совмещена. Заполним пропуски нулём.

In [ ]:
# Используем метод fillna для заполнения оставшихся пропущенных значений в колонке kitchen_area 
df['kitchen_area'] = df['kitchen_area'].fillna(0)

In [ ]:
# Снова проверим количество пропущенных значений в колонке kitchen_area
print('Количество отсутствующих значений -', df['kitchen_area'].isna().sum())

В колонке `balcony` -  число балконов, 11,519 пропущенных значений. Если продавец не указал число балконов, то, скорее всего, балкон отсутствует. Заменим данные пропуски нулём.

In [ ]:
# Используем метод fillna для заполнения пропущенных значений в колонке balcony
df['balcony'] = df['balcony'].fillna(0)

In [ ]:
# Проверим количество пропущенных значений в колонке balcony
print('Количество отсутствующих значений -', df['balcony'].isna().sum())

Преобразуем тип данных в столбцах: 
- `last_price`,
- `floors_total`,
- `balcony`.

In [ ]:
# Преобразуем колонки в целочисленный тип данных int используя метод astype
df[['last_price', 'floors_total', 'balcony']] = df[['last_price', 'floors_total', 'balcony']].astype(int)

Вывод: 

- для расчета жилой площади выводилось медианное значение доли жилой площади в общей, в зависимости от количества комнат в квартире.
- площадь кухни бралась как медиана в зависимости от количества комнат.
- в значениях высоты потолков убрали выбросы менее 2 метров и более 7 метров, заменив их медианой.
- пропуски в значениях аппартаментов заменили на False.
- пропуски в количестве балконов заменили на 0.
- ~~количество этажей в доме приравняли этажу квартиры.~~ удалили пропущенные значения
- изменили типы данных у цены, этажей и балконов на целое значение.

Все пропуски в колонках объектов недвижимости обработаны, пропущенных значений - нет.

---

Изучим следующие параметры объектов:
- количество объектов;
- расстояние до центра города;
- расстояние до ближайшего аэропорта.

Обработаем пропущенные значения.

In [ ]:
# Выведем количество населённых пунктов
df['locality_name'].nunique()

In [ ]:
# Выведем количество объектов для продажи в населённых пунктов в порядке убывания
df['locality_name'].value_counts()

В таблице представлено 364 населенных пункта, относящихся к г. Санкт-Петербург и Ленинградской области.   
В колонке locality_name - название населённого пункта, обнаружено 49 пропущенных значений. Для их заполнения посмотрим насколько они удалены от центра города.

In [ ]:
# Выведем описательную статистику с использованием среза данных c операцией сравнения
# для объектов Ленинградской области
df.query('locality_name != locality_name')['city_centers_nearest'].describe()

In [ ]:
# Выведем описательную статистику с использованием среза данных c операцией сравнения
# для объектов г. Санкт-Петербург
df.query('locality_name == "Санкт-Петербург"')['city_centers_nearest'].describe()

Пропущенные значения могут относиться как к объектам в городе так и к объектам в области. Предположим что, если пропуск находится ближе чем медианное значение расстояния от центра города, то это г. Санкт-Петербург, остальным пропущенным значениям выставим - "прочие локации".

In [ ]:
# Создадим переменную в которую положим медианное значение
city_centers_nearest_median = df.query('locality_name == "Санкт-Петербург"')['city_centers_nearest'].median()

In [ ]:
def distance(city_centers_nearest):
    """Функция сравнивает расстояние до центра города
    и возвращает полученное значение
    """
    if city_centers_nearest < city_centers_nearest_median:
        return 'Санкт-Петербург'
    return 'прочие локации'

In [ ]:
# Cоздадим отдельный столбец и в его ячейках запишем значения, возвращаемые функцией.
df['locality_name_values'] = df['city_centers_nearest'].apply(distance)

In [ ]:
# Положим в переменную результат работы функции для заполнения пропущенных значений
df['locality_name'] = df['locality_name'].fillna(df['locality_name_values'])

In [ ]:
# Удалим ставшую ненужной колонку locality_name_values методом dropna
df = df.drop(columns=['locality_name_values'], axis=1)

In [ ]:
# Используя логическую индексацию, подсчитаем количество оставшихся значений "прочие локации"
df[df['locality_name'] == 'прочие локации']['rooms'].count()

В колонкае `airports_nearest` - расстояние до ближайшего аэропорта в метрах (м), 5,542 пропущенных значения; в колонке `city_centers_nearest` - расстояние до центра города (м), 5,519 пропущенных значений. Посмотрим, сколько пропущенных значений в самом большом городе Ленинградской области - Санкт-Петербурге.

In [ ]:
# Используем срез данных c операцией сравнения для подсчёта пропущенных значений
df.query('locality_name == "Санкт-Петербург"')['airports_nearest'].isna().sum()

In [ ]:
# Используем срез данных c операцией сравнения для подсчёта пропущенных значений
df.query('locality_name == "Санкт-Петербург"')['city_centers_nearest'].isna().sum()

Количество пропущенных значений в обоих случаях не велико, поэтому заменим их на медианное значение соответствующего населённого пункта

In [ ]:
# Используем метод fillna c аргументом групировки: населённый пункт - медианное значение
df['airports_nearest'] = (
    df['airports_nearest'].fillna(df
                                    .groupby('locality_name')['airports_nearest']
                                    .transform('median'))
)

df['city_centers_nearest'] = (
    df['city_centers_nearest'].fillna(df
                                        .groupby('locality_name')['city_centers_nearest']
                                        .transform('median'))
)

Переведём расстояние до центра города в километры и округлим до целых значений.

In [ ]:
# Округлим до целых значений и положим в ту же самую колонку
df['city_centers_nearest'] = (df['city_centers_nearest'] / 1000).round(0)

Вывод:

- ввиду малого количества пропущенных значений удаленности от аэропорта и от центра г. Санкт-Петербург, заменили их на соответствующие медианные значения. Остальные пропущенные значения относятся к другим населенным пунктам расположенными в Ленинградской области, их не трогали.

В колонках `parks_around_3000` - число парков в радиусе 3 км и `ponds_around_3000` -  число водоёмов в радиусе 3 км, по 5,518 пропущенных значений. Видимо между столбцами существует определённая взаимосвязь. Заменим пропуски на 0 и изменим тип данных на `int`.

In [ ]:
# Заменим пропуски нулём
df[['parks_around_3000', 'ponds_around_3000']] = df[['parks_around_3000', 'ponds_around_3000']].fillna(0)

# Изменим тип данных на int
df[['parks_around_3000', 'ponds_around_3000']] = df[['parks_around_3000', 'ponds_around_3000']].astype(int)

Вывод:

- из 23699 строк таблицы 15747(66,4%) приходится на Санкт-Петербург. Заменили пропущенные значения на медианные. Значительная часть пропущенных значений осталась не заполнена, но так как эти пропуски в основном относятся к объектам недвижимости Ленинградской области, то расстояние до центра и аэропорта не столь актуально;
- из 49 пропущенных значений в колонке locality_name, заменили 26 значений на г. Санкт-Петербург и 23 значения на "прочие локации";
- для заполнения пропущенных значений, можно воспользоваться сервисом Яндекс.карты и выписать какое поселение относится к городу, а какое к области, измерить расстояние до центра города и прочие показатели;
- частично были изменены типы данных, метры заменили на километры с округлением до целой части.

---

Изучим уникальные значения в столбце с названиями населённых пунктов и устраним неявные дубликаты. Например, «поселок Рябово» и «поселок городского типа Рябово», «поселок Тельмана» и «посёлок Тельмана» — это обозначения одних и тех же населённых пунктов. Заменим названия в существующем столбце или создадим новый с названиями без дубликатов.

In [ ]:
# Выведем на экран перечень уникальных названий населённых пунктов
df['locality_name'].unique()

Много населенных пунктов, но некоторые из них имеют только одно значение.

Кроме того, из названий городов и поселков видно, что в основном все они расположены вокруг Санкт-Петербурга. Можно понять размер населенного пункта по его названию: "деревня", "поселок", "поселок городского типа" и так далее. Города записываются просто по названиям, без дополнительных уточнений.

Чтобы облегчить анализ данных, оставим только названия населенных пунктов и лемматизируем их для разделения на категории.

Создадим функцию лемматизации, которая на основании данных из столбца `locality_name` сформирует новый столбец `locality_name_lem`, в который войдут следующие локации:

In [ ]:
def lemm(liveplace):
    """Функция  принимает текст и возвращает 
    лемматизированную строку
    """
    lemma = ' '.join(m.lemmatize(liveplace))
    if "поселок город тип" in lemma:
        return lemma.replace("посёлок город тип", '')
    elif "деревня" in lemma:
        return lemma.replace("деревня", '')
    elif "поселок" in lemma:
        return lemma.replace("поселок", '')
    elif "село" in lemma:
        return lemma.replace("село", '')
    elif "товарищество" in lemma:
         return lemma.replace("товарищество", '')
    else:
        pass
    return lemma

In [ ]:
#  Применим к таблице df и добавим результат в locality_name_lem
df["locality_name_lem"] = df["locality_name"].apply(lemm)

In [ ]:
# Проверим результат
df["locality_name_lem"].value_counts()

Вывод: 
- изучили уникльные значения населённых пунктов;
- для облегчения анализа, упростили названия населённых пунктов;
- устранили неявные дубликаты в населённых пунктах используя лемматизацию.

---

**Исследуем на выбросы `last_price` - цена**

In [ ]:
df.boxplot(column='last_price')
plt.ylim(10_000, 500_000)
plt.show()

df.boxplot(column='last_price')
plt.ylim(12_000_000, 800_000_000)
plt.show()

Посмотрим на выбросы за границами значений, которые не вошли в "ящик с усами"

In [ ]:
len(df.query('last_price > 200_000_000'))

9 выбивающихся значений - удалим их.


In [ ]:
len(df)

In [ ]:
# Отфильтруем выбросы методом query
df = df.query('last_price < 200_000_000')

Уберем выбросы за пределами нормальных значений

In [ ]:
describe_last_price = df['last_price'].describe().T
q1_last_price = describe_last_price['25%'] # первый квартиль
q3_last_price = describe_last_price['75%'] # третий квартиль
iqr_last_price = q3_last_price - q1_last_price  # межквартильный размах

# уберём выбросы за пределами нормальных значений
df = df.query('(@q1_last_price-1.5*(@iqr_last_price)) <= last_price <= (@q3_last_price+1.5*(@iqr_last_price))')

# Отображение диаграммы boxplot
df[['last_price']].boxplot()


In [ ]:
print(describe_last_price)
len(df)

**Исследуем на выбросы total_area — общая площадь квартиры в квадратных метрах (м²)**

In [ ]:
df.boxplot(column='total_area')

In [ ]:
df.boxplot(column='total_area')
plt.ylim(0, 50)
plt.show()

In [ ]:
df.boxplot(column='total_area')
plt.ylim(150, 500)
plt.show()

In [ ]:
len(df.query('total_area > 150'))

16 выбивающихся значений - удалим их. 

In [ ]:
df = df.query('total_area < 150')

In [ ]:
len(df)

Уберем выбросы за пределами нормальных значений

In [ ]:
describe_total_area = df['total_area'].describe().T
q1_total_area = describe_total_area['25%'] # первый квартиль
q3_total_area = describe_total_area['75%'] # третий квартиль
iqr_total_area = q3_total_area - q1_total_area  # межквартильный размах

# уберём выбросы за пределами нормальных значений
df = df.query('(@q1_total_area-1.5*(@iqr_total_area)) <= total_area <= (@q3_total_area+1.5*(@iqr_total_area))')

# Отображение диаграммы boxplot 
df[['total_area']].boxplot()

In [ ]:
print(describe_total_area)
len(df)

**Исследуем на выбросы `rooms` — число комнат**

In [ ]:
df.boxplot(column='rooms')

In [ ]:
df.boxplot(column='rooms')
plt.ylim(5, 20)
plt.show()

In [ ]:
df.boxplot(column='rooms')
plt.ylim(0, 1)
plt.show()

In [ ]:
df.query('rooms == 0')

Похоже, что это студии, так как данные объекты имеют общую и жилую площадь. Не будем их трогать.

Уберем выбросы за пределами нормальных значений

In [ ]:
describe_rooms = df['rooms'].describe().T
q1_rooms = describe_rooms['25%'] # первый квартиль
q3_rooms = describe_rooms['75%'] # третий квартиль
iqr_rooms = q3_rooms - q1_rooms  # межквартильный размах

# уберём выбросы за пределами нормальных значений
df = df.query('(@q1_rooms-1.5*(@iqr_rooms)) <= rooms <= (@q3_rooms+1.5*(@iqr_rooms))')

# Отображение диаграммы boxplot 
df[['rooms']].boxplot()

In [ ]:
print(describe_rooms)
len(df)

**Исследуем на выбросы `days_exposition` — сколько дней было размещено объявление (от публикации до снятия)**

In [ ]:
df.boxplot(column='days_exposition')

In [ ]:
df.boxplot(column='days_exposition')
plt.ylim(1400, 5_000)
plt.show()

In [ ]:
len(df.query('days_exposition > 1400'))

In [ ]:
df = df.query('days_exposition < 1400')

Уберем выбросы за пределами нормальных значений

In [ ]:
describe_days_exposition = df['days_exposition'].describe().T
q1_days_exposition = describe_days_exposition['25%'] # первый квартиль
q3_days_exposition = describe_days_exposition['75%'] # третий квартиль
iqr_days_exposition = q3_days_exposition - q1_days_exposition  # межквартильный размах

# уберём выбросы за пределами нормальных значений
df = df.query('(@q1_days_exposition-1.5*(@iqr_days_exposition)) <= days_exposition <= (@q3_days_exposition+1.5*(@iqr_days_exposition))')

# Отображение диаграммы boxplot 
df[['days_exposition']].boxplot()

In [ ]:
print(describe_days_exposition)
len(df)

### Посчитайте и добавьте в таблицу новые столбцы

Добавим колонку с расчётом цены за метр квадратный

In [ ]:
# Частное от цены на площадь положим в price_square_meter
df['price_square_meter'] = df['last_price'] / df['total_area']
# Изменим тип данных частного на int
df['price_square_meter'] = df['price_square_meter'].astype(int)

Добавим колонки с днём недели, месяцем и годом

In [ ]:
# Используем атрибут dt (date time), который укажет, что тип 
# данных, к которым будут применены методы, - datetime
df['day_of_the_week'] = df['first_day_exposition'].dt.weekday
df['month'] = df['first_day_exposition'].dt.month
df['year'] = df['first_day_exposition'].dt.year

Используем функцию для добавления типа этажа квартиры

In [ ]:
def floor_type(row):
    """В тело функции вложим конструкцию
    с условием истинности
    """
    floor = row['floor']
    flors_total = row['floors_total']
    
    if floor == 1:
        return 'первый'
    elif floor == flors_total:
        return 'последний'
    else:
        return 'другой'   

In [ ]:
# Подставим результат работы функции в колонку floor_type
df['floor_type'] = df.apply(floor_type, axis=1)

In [ ]:
df.head()

Добавим колонки:
- отношение общей площади к жилой площади;
- отношение площади кухни к общей площади.

In [ ]:
# Результат вычислений положим в переменные living_total_area и kitchen_total_area
df['living_total_area'] = df['living_area'] / df['total_area']
df['kitchen_total_area'] = df['kitchen_area'] / df['total_area']

In [ ]:
# Проверим результат вычислений в колонках living_total_area и kitchen_total_area
df.head()

---

### Проведите исследовательский анализ данных

**Изучим данные колонки `total_area` - площадь квартиры в квадратных метрах (м²).**

In [ ]:
df['total_area'].plot(kind='hist', title='Площадь квартир (м²):', figsize=(10, 5), grid=True)
plt.show()

In [ ]:
df['total_area'].describe()

Видно, что преобладающее число объявлений имеют площадь до 100 м², затем идет резкое снижение и далее значение практически сливаются с нулем.

Посмотрим на диапазоны значений:

In [ ]:
df['total_area'].plot(kind='hist'
                      , title='Площадь квартир 0-40 (м²):'
                      , figsize=(10, 5)
                      , range=(0, 40)
                      , bins=100, grid=True
                     )
plt.show()
print(df.query('total_area < 40')['total_area'].describe())


df['total_area'].plot(kind='hist'
                      , title='Площадь квартир 41-60 (м²):'
                      , figsize=(10, 5)
                      , range=(40, 60)
                      , bins=100
                      , grid=True
                     )
plt.show()
print(df.query('41 < total_area < 60')['total_area'].describe())


df['total_area'].plot(kind='hist'
                      , title='Площадь квартир 61-100 (м²):'
                      , figsize=(10, 5)
                      , range=(60, 100)
                      , bins=100
                      , grid=True
                     )
plt.show()
print(df.query('61 < total_area')['total_area'].describe())

Вывод

Видим аномальное минимальное значение 12 м² - точно не квартира,слишком маленькая площадь, может быть это комната или ошибка в данных.

В среднем, площадь квартир составляет 51 м² и это не сильно отличается от медианы 47 м². На гистограмме видно, что основная доля квартир приходится на площадь 40 - 60 м².

**Изучим данные колонки `living_area` - жилая площадь в квадратных метрах (м²).**

In [ ]:
df['living_area'].plot(kind='hist'
                       , title='Жилая площадь квартир (м²):'
                       , figsize=(10, 5)
                       , grid=True
                      )
plt.show()

In [ ]:
df['living_area'].describe()

Видно, что преобладающее число объявлений имеют площадь до 60 м², затем идет резкое снижение до 90 м² и далее значение практически сливаются с нулем.

Взглянув на числовое описание данных, видим, что третий квартиль и максимальное значение сильно различаются: 37 и 88, а так же первый квартиль и минимальное значение - 2 и 18.

Посмотрим на эти диапазоны:

In [ ]:
df['living_area'].plot(kind='hist'
                       , title='Жилая площадь квартир 0-30 (м²):'
                       , figsize=(10, 5)
                       , range=(0, 30)
                       , bins=100
                       , grid=True
                      )
plt.show()
print(df.query('living_area < 31')['living_area'].describe())


df['living_area'].plot(kind='hist'
                       , title='Жилая площадь квартир 31-60 (м²):'
                       , figsize=(10, 5)
                       , range=(30, 60)
                       , bins=100
                       , grid=True
                      )
plt.show()
print(df.query('30 < living_area < 61')['living_area'].describe())


df['living_area'].plot(kind='hist'
                       , title='Жилая площадь квартир 61-90 (м²):'
                       , figsize=(10, 5)
                       , range=(60, 90)
                       , bins=100
                       , grid=True
                      )
plt.show()
print(df.query('60 < living_area')['living_area'].describe())

Вывод

Видим аномальное минимальное значение 2 м² - слишком маленькая жилая площадь, вероятнее всего, это ошибка в данных. Также мы видим резкий спад после 60 м² и очень маленькое количество объявлений, всего 148 шт., на фоне всей выборке это менее 1% данных.

На гистограмме видно, что основная доля приходится на квартиры с жилой площадью до 30 м².

**Изучим данные колонки `kitchen_area` - площадь кухни в квадратных метрах (м²).**

In [ ]:
df['kitchen_area'].plot(kind='hist'
                        , title='Площадь кухни (м²):'
                        , figsize=(10, 5)
                        , grid=True)
plt.show()

In [ ]:
df['kitchen_area'].describe()

Видно, что преобладающее число объявлений имеют площадь кухни до 20 м², затем идет резкое снижение до 30 м² и далее значение практически сливаются с нулем.

Взглянув на числовое описание данных, видим, что третий квартиль и максимальное значение сильно различаются: 10 и 50, а так же первый квартиль и минимальное значение - 0 и 7.

Посмотрим на эти диапазоны:

In [ ]:
df['kitchen_area'].plot(kind='hist'
                        , title='Площадь кухни 0-20 (м²):'
                        , figsize=(10, 5)
                        , range=(0, 20)
                        , bins=100
                        , grid=True
                       )
plt.show()
print(df.query('kitchen_area < 21')['kitchen_area'].describe())


df['kitchen_area'].plot(kind='hist'
                        , title='Площадь кухни 21-50 (м²):'
                        , figsize=(10, 5)
                        , range=(20, 50)
                        , bins=100
                        , grid=True
                       )
plt.show()
print(df.query('20 < kitchen_area < 51')['kitchen_area'].describe())

Вывод

Видим аномальное минимальное значение 0 м² - это квартира студия. 

На гистограмме видно, что основная доля приходится на квартиры с кухней до 20 м².

**Изучим данные колонки `last_price` - цена на момент снятия с публикации, c выводом гистограммы и сортировкой значений**

In [ ]:
df['last_price'].plot(kind='hist'
                      , title='Стоимость квартир'
                      , figsize=(10, 5)
                      , bins=100
                      , grid=True
                     )
plt.show()

In [ ]:
df['last_price'].describe()

- Минимальная стоимость квартиры: 430,000.00.
- Максимальная стоимость квартиры: 11,850,000.00.

Разобьем на три диапазона по стоимости:

- от 0 до 3,000,000 рублей;
- от 3,000,000 рублей до 6,000,000;
- выше 6,000,000.

In [ ]:
# до 3 млн. рублей
(
df['last_price']
    .plot(kind='hist' 
         , title='Стоимость квартир до 6 млн. рублей'
         , range=(0, 3_000_000)
         , figsize=(10, 5)
         , bins=100
         , grid=True        
         )
)

plt.show()
print(df.query('last_price < 3_000_000')['last_price'].describe())



# от 3 млн. рублей до 6 млн. рублей
(
df['last_price']
    .plot(kind='hist' 
         , title='Стоимость квартир от 3 млн. рублей до 6 млн. рублей' 
         , range=(3_000_001, 6_000_000)
         , figsize=(10,5)
         , bins=100
         , grid=True        
         )
)

plt.show()
print(df.query('3_000_001 < last_price < 6_000_000')['last_price'].describe())



# от 6 млн. рублей
(
df['last_price']
    .plot(kind='hist' 
         , title='Стоимость квартир от 6 млн. рублей' 
         , range=(6_000_000, 12_000_000)
         , figsize=(10,5)
         , bins=100
         , grid=True        
         )
)

plt.show()
print(df.query('last_price > 6_000_000')['last_price'].describe())


Видно небольшое стандартное отклонение у квартир до 3 млн. рублей, и далее у квартир следющих диапазонов.

Большего всего квартир продаётся от 3 до 6 млн. рублей, затем гистограмма идет на спад.

Вывод

Медиана цены приходится на 4,290,000.00, среднее значение на 4,650,043.71, стандартное отклонение на 2,043,696.73. Отклонение, является результатом высоких цен за объекты, самая высокая из которых - 11,850,000.00.

---


**Изучим данные колонки `rooms` - число комнат, c выводом гистограммы и сортировкой значений**

In [ ]:
df['rooms'].plot(kind='hist'
                 , title='Количество комнат'
                 , bins=50
                 , figsize=(10, 5)
                 , grid=True
                )

plt.show()

In [ ]:
df['rooms'].describe()

Видим количество комнат равное 0 - ранее мы предположили, что это квартиры-студии. Стандартное отклонение стремится к 1, медиана и среднее арифметическое практически равны.

На гистограмме наблюдаем, что в большинстве объявлений количество комнат равняется: 1, 2 или 3. Начиная с 4-х комнатных квартир мы видим спад.

Вывод

Самые популярные одно - и двухкомнатные квартиры, далее по убыванию.

---

**Изучим данные колонки `ceiling_height` - высота потолков (м).**

In [ ]:
df['ceiling_height'].plot(kind='hist'
                          , title='Высота потолков в квартире (м):'
                          , figsize=(10, 5)
                          , bins=100
                          , grid=True)
plt.show()

In [ ]:
df['ceiling_height'].describe()

Cтандартная высота потолка в квартирах  2,5-3,2 метра в зависимости от года строительства и типа дома. В некоторых домах "старой застройки" (сталинские дома) высота потолка может достигать – от 3,20 м до 4 - 6 м, посмотрим на гистограмму таких значений:

In [ ]:
df.query('2.5 <= ceiling_height <= 6')['ceiling_height'].hist()
plt.show()

In [ ]:
df.query('2.5 <= ceiling_height <= 6')['ceiling_height'].describe()

16,910 значений соответствует нашему диапазону. Стандартное отклонение стремится к нулю, медиана и среднее арифметическое почти равны.

Вывод

Самые распространенное значение высоты потолка - 2.65 метра, что соответствует медиане.

---

**Изучим данные колонки `floor` - этаж квартиры.**

In [ ]:
df['floor'].plot(kind='hist'
                 , title='Этаж квартиры'
                 , bins=50
                 , figsize=(10, 5)
                 , grid=True
                )

plt.show()

In [ ]:
df['floor'].describe()

К минимальному значению и 25-у перцентилю вопросов нет, а вот 75 перцентиль и максимальное значение нужно посмотреть. Разделим данные на три диапазона и посмотрим их:

In [ ]:
df.head()

In [ ]:
df['floor'].plot(kind='hist'
                 , title='Номер этажа 1 - 5:'
                 , figsize=(10, 5)
                 , range=(0, 5)
                 , bins=100
                 , grid=True
                )
plt.show()


df['floor'].plot(kind='hist'
                 , title='Номер этажа 6 - 15:'
                 , figsize=(10, 5)
                 , range=(5, 15)
                 , bins=100
                 , grid=True
                )
plt.show()


df['floor'].plot(kind='hist'
                 , title='Номер этажа 16 - 33:'
                 , figsize=(10, 5)
                 , range=(15, 40)
                 , bins=100
                 , grid=True
                )
plt.show()

Вывод

Наибольшее количество объявлений приходится на диапазон с 1 по 5 этаж, далее на диапазон с 6 по 15 этаж и наконец на этажи с 16 по 33. Средние и медианные значения почти совпадают.

---

**Изучим данные колонки `floor_type` - тип этажа квартиры («первый», «последний», «другой»).**

In [ ]:
df['floor_type'].value_counts()

In [ ]:
df['floor_type'].value_counts().sum()

Вывод

Наибольшую долю (74%) от всех объявлений, составляет тип этажа "другой", далее с долей 14% тип этажа "последний" и на третьем месте с долей 12% тип этажа "первый".

**Изучим данные колонки `floors_total` - всего этажей в доме.**

In [ ]:
df['floors_total'].plot(kind='hist'
                        , title='Этаж квартиры'
                        , bins=50
                        , figsize=(10, 5)
                        , grid=True
                       )
plt.show()

In [ ]:
df['floors_total'].describe()

Видим, что наибольшее количество объявлений приходится на дома с высотой до 10 этажа. Сделаем срезы, посмотрим повнимательнее:

In [ ]:
df['floors_total'].plot(kind='hist'
                        , title='Этажность дома 1 - 10:'
                        , figsize=(10, 5)
                        , range=(1, 10)
                        , bins=100
                        , grid=True
                       )
plt.show()
print(df.query('floors_total < 11')['floors_total'].describe())


df['floors_total'].plot(kind='hist'
                        , title='Этажность дома 11 - 20:'
                        , figsize=(10, 5)
                        , range=(10, 20)
                        , bins=100
                        , grid=True
                       )
plt.show()
print(df.query('10 < floors_total < 21')['floors_total'].describe())


df['floors_total'].plot(kind='hist'
                        , title='Этажность дома 21 - 60:'
                        , figsize=(10, 5)
                        , range=(20, 60)
                        , bins=100
                        , grid=True
                       )
plt.show()
print(df.query('20 < floors_total')['floors_total'].describe())


In [ ]:
df.query('floors_total == 60')

Вывод:

- в первом сегменте наибольшая доля объявлений приходится на 5-и этажные дома, далее с небольшим отрывом на 9-и этажные дома;
- во втором сегменте наибольшая доля объявлений приходится на 12-и и 16-и этажные дома;
- в третьем сегменте на 25-и этажные дома.

Максимальное значение в 60 этажей, похоже на ошибку разрядности, должно быть 6 этажей.

**Изучим данные колонки `city_сenter_nearest` - расстояние до центра города (м).**

In [ ]:
df['city_centers_nearest'].plot(kind='hist'
                                , title='Расстояние до центра города (км)'
                                , bins=50
                                , figsize=(10, 5)
                                , grid=True
                               )
plt.show()

In [ ]:
df['city_centers_nearest'].describe()

На гистограмме, видим три пиковых значения размещения объявлений квартир относительно центра города. Разделим данные на сегменты и изучим показатели:

In [ ]:
df['city_centers_nearest'].plot(kind='hist'
                                , title='Расстояние до центра города (км) 0 - 10:'
                                , figsize=(10, 5)
                                , range=(0, 10)
                                , bins=100
                                , grid=True
                               )
plt.show()
print(df.query('city_centers_nearest <= 10')['city_centers_nearest'].describe())


df['city_centers_nearest'].plot(kind='hist'
                                , title='Расстояние до центра города (км) 11 - 20:'
                                , figsize=(10, 5)
                                , range=(11, 20)
                                , bins=100
                                , grid=True
                               )
plt.show()
print(df.query('11 < city_centers_nearest < 20')['city_centers_nearest'].describe())


df['city_centers_nearest'].plot(kind='hist'
                                , title='Расстояние до центра города (км) 21 - 30:'
                                , figsize=(10, 5)
                                , range=(21, 30)
                                , bins=100
                                , grid=True
                               )
plt.show()
print(df.query('21 < city_centers_nearest < 30')['city_centers_nearest'].describe())


df['city_centers_nearest'].plot(kind='hist'
                                , title='Расстояние до центра города (км) 31 - 70:'
                                , figsize=(10, 5)
                                , range=(31, 70)
                                , bins=100
                                , grid=True)
plt.show()
print(df.query('31 < city_centers_nearest')['city_centers_nearest'].describe())


Вывод:
 - в сегменте до 10,000 метров (м) от центра, наибольшее количество объявлений приходится на квартиры расположенные в 10,000 метрах;
 - в сегменте от 11,000 м до 20,000 м от центра, наибольшее количество объявлений приходится на квартиры расположенные в 12,000 метрах;
 - в сегменте от 21,000 до 30,000 метров от центра, наибольшее количество объявлений приходится на квартиры расположенные в 22,000 и 30,000 метрах;
 - в сегменте 31,000 до 70,000 метров от центра, наибольшее количество объявлений приходится на квартиры расположенные в 31,000 метрах;
 
В первых трёх сегментах, средняя и медиана почти совпадают. В четвёртом сегменте средняя и медиана немного расходятся из-за высокого максимального значения - 66,000 метров.


**Изучим данные колонки `airports_nearest` - расстояние до ближайшего аэропорта в метрах (м).**

In [ ]:
df['airports_nearest'].plot(kind='hist'
                            , title='Расстояние до ближайшего аэропорта (м)'
                            , bins=100
                            , figsize=(10, 5)
                            , grid=True
                           )
plt.show()

In [ ]:
df['airports_nearest'].describe()

In [ ]:
df.query('airports_nearest == 0')

Квартира с нулевым минимальным значением - скорее всего ошибка заполнения, либо находися около аэропорта.

Наибольшее количество объявлений о продаже квартир размещено от 10,000 до 40,000 метров. Разделим данные на сегменты, изучим показатели:

In [ ]:
df['airports_nearest'].plot(kind='hist'
                            , title='Расстояние до ближайшего аэропорта в метрах (м) 0 - 15,000:'
                            , figsize=(10, 5)
                            , range=(0, 15000)
                            , bins=100
                            , grid=True
                           )
plt.show()
print(df.query('airports_nearest < 15000')['airports_nearest'].describe())


df['airports_nearest'].plot(kind='hist'
                            , title='Расстояние до ближайшего аэропорта в метрах (м) 15,000 - 30,000:'
                            , figsize=(10, 5)
                            , range=(16000, 30000)
                            , bins=100
                            , grid=True
                           )
plt.show()
print(df.query('15000 < airports_nearest < 30000')['airports_nearest'].describe())


df['airports_nearest'].plot(kind='hist'
                            , title='Расстояние до ближайшего аэропорта в метрах (м) 30,000 - 45,000:'
                            , figsize=(10, 5)
                            , range=(31000, 45000)
                            , bins=100
                            , grid=True)
plt.show()
print(df.query('30000 < airports_nearest < 45000')['airports_nearest'].describe())


df['airports_nearest'].plot(kind='hist'
                            , title='Расстояние до ближайшего аэропорта в метрах (м) 45,000 - 85,000:'
                            , figsize=(10, 5)
                            , range=(46000, 85000)
                            , bins=100
                            , grid=True)
plt.show()
print(df.query('45000 < airports_nearest')['airports_nearest'].describe())


Вывод:
 - наибольше количество объявлений размещено в сегменте 15,000 - 30,000 м - 4,978;
 - на втором месте, количество объявлений размещено в сегменте 30,000 - 45,000 м - 3,931;
 - на третьем месте, количество объявлений размещено в сегменте 0 - 15,000 м - 2,215;
 - на четвёртом месте, количество объявлений размещено в сегменте 45,000 - 85,000 м - 1,779
 
 Средние и медианные значение почти везде совпадают за небольшим исключением в четвёртом сегменте.

**Изучим данные колонки `parks_nearest` - расстояние до ближайшего парка (м).**

In [ ]:
df['parks_nearest'].plot(kind='hist'
                         , title='Расстояние до ближайшего парка (м)'
                         , bins=100
                         , figsize=(10, 5)
                         , grid=True
                        )
plt.show()

In [ ]:
df['parks_nearest'].describe()

In [ ]:
df.query('parks_nearest == 1')

In [ ]:
df.loc[14563]

Объект с минимальным значением = 1, скорее всего, ошибка заполнения.

Вывод:

- наибольшее количество объявлений сосредоточено в 1,000 от парка. Средня и медиана вблизи значений друг друга.
- сегментировать 1,000 м, думаю нет необходимости

**Изучим данные колонки `day_of_the_week` - день размещения объявления.**

In [ ]:
df['day_of_the_week'].plot(kind='hist'
                           , title='День размещения объявления'
                           , bins=100
                           , figsize=(10, 5)
                           , grid=True)
plt.show()

In [ ]:
df['day_of_the_week'].value_counts()

Вывод:
 - наибольшее количество размещеня объявлений в четверг - 3,078;
 - на втором месте, вторник - 2,969;
 - на третьем месте, пятница - 2,966.
    
Меньше всего размещений в субботу и воскресене, так как люди отдыхают.

**Изучим данные колонки `month` - месяц размещения объявления.**

In [ ]:
df['month'].plot(kind='hist'
                 , title='Месяц размещения объявления'
                 , bins=100
                 , figsize=(10, 5)
                 , grid=True)
plt.show()

In [ ]:
df['month'].value_counts()

Вывод:
 - самое большое количество объявлений размещено в феврале - 1,935;
 - на втором месте, ноябрь - 1800;
 - на третьем, октябрь - 1662.
 
Меньше всего объявлений размещено в мае и январе, видимо это связано с длинными майским и новогодними празниками. 

Изучим данные колонки `days_exposition` - сколько дней было размещено объявление (от публикации до снятия), c выводом гистограммы и сортировкой значений

In [ ]:
df['days_exposition'].describe()

In [ ]:
df['days_exposition'].hist(bins=30,figsize=(10,5))

plt.title("Распределение продажи квартир")
plt.xlabel("Количество дней")
plt.ylabel("Количество квартир")

In [ ]:
df['days_exposition'].value_counts().head()

Вывод

Большое стандартное отклонение, что свидетельствует о долгих продажах объектов.

Больше всего объектов продано за 45 дней или на уровне 25-го перцентиля. Всё что ниже этой отметки, можно отнести к очень быстрой продаже и наоборот, всё что выше 75-го перцентиля, очень долгой продаже. 

---

Изучим зависимость цены продажи объекта - `last_price` от площади объекта - `total_area`.

In [ ]:
df_corr_lp_tp = df[['last_price', 'total_area']]

df.plot(x='total_area', y='last_price', kind='scatter', grid=True,figsize=(10, 5))
plt.title("Зависимость цены объекта от общей площади")
plt.xlabel("Общая площадь")
plt.ylabel("Цена объекта")

In [ ]:
# Применим метод корреляции Пирсона
df['last_price'].corr(df['total_area']).round(2)

Корреляция в 0.68 говорит о достаточной связи, однако не слишком сильной.

**Изучим зависимость цены продажи объекта - `last_price` от количества комнат - `rooms`.**

In [ ]:
df['last_price'].corr(df['rooms']).round(2)

In [ ]:
df.plot(x='rooms'
        , y='last_price'
        , kind='scatter'
        , grid=True
        , figsize=(10, 5)
       )

plt.title("Зависимость цены объекта от числа комнат")
plt.xlabel("Число комнат")
plt.ylabel("Цена объекта")

Корреляция в - 0.42 говорит о недостаточной связи.

**Изучим зависимость цены продажи объекта - `last_price` от расстояние до центра города (м) - `city_сenters_nearest`.**

In [ ]:
df['last_price'].corr(df['city_centers_nearest']).round(2)

In [ ]:
df.plot(x='city_centers_nearest'
        , y='last_price'
        , kind='scatter'
        , grid=True
        , figsize=(10, 5)
       )

plt.title('Зависимость цены объекта от близости к центру')
plt.xlabel('Расстояние до центра (км)')
plt.ylabel('Цена объекта')

Видим обратную зависимость, чем меньше расстояние от центра, тем больше цена объекта.

**Изучим зависимость цены продажи объекта - `last_price` от этажа - `floor`.**

In [ ]:
df.groupby('floor_type')['last_price'].median()


In [ ]:
df.plot(x='floor'
        , y='last_price'
        , kind='scatter'
        , grid=True
        , figsize=(10, 5)
       )

plt.title('Зависимость цены объекта от номера этажа')
plt.xlabel('Номер этажа')
plt.ylabel('Цена объекта')

Самая низкая цена на первые этажи, дороже последние и самая дорогая категория между первым и последним этажем.

**Изучим зависимость цены продажи объекта - `last_price` от дня недели публикации - `day_of_the_week`.**

In [ ]:
df.groupby('day_of_the_week')['last_price'].median()

In [ ]:
df.plot(x='day_of_the_week'
        , y='last_price'
        , kind='scatter'
        , grid=True
        , figsize=(10, 5)
       )

plt.title('Зависимость цены объекта от дня недели')
plt.xlabel('День недели')
plt.ylabel('Цена объекта')

Зависимость отсутствует

**Изучим зависимость цены продажи объекта - `last_price` от месяца публикации - `month`.**

In [ ]:
df.groupby('month')['last_price'].median()

In [ ]:
df.plot(x='month'
        , y='last_price'
        , kind='scatter'
        , grid=True
        , figsize=(10, 5)
       )

plt.title('Зависимость цены объекта от месяца')
plt.xlabel('Месяц')
plt.ylabel('Цена объекта')

Зависимость отсутствует, в пределах небольшой погрешности.

**Изучим зависимость цены продажи объекта - `last_price` от года публикации - `year`.**

In [ ]:
df.groupby('year')['last_price'].median()


In [ ]:
df.plot(x='year'
        , y='last_price'
        , kind='scatter'
        , grid=True
        , figsize=(10, 5)
       )

plt.title('Зависимость цены объекта от года')
plt.xlabel('Год')
plt.ylabel('Цена объекта')

Самые высокие цены были в 2019 году, самые низкие в 2016, 2017 годах.

**Изучим зависимость цены продажи объекта - `last_price` от жилой площади - `living_area`.**

In [ ]:
df['last_price'].corr(df['living_area']).round(2)

In [ ]:
df.plot(x='living_area'
        , y='last_price'
        , kind='scatter'
        , grid=True
        , figsize=(10, 5)
       )

plt.title('Зависимость цены объекта от жилой площади')
plt.xlabel('Жилая площадь')
plt.ylabel('Цена объекта')

Корреляция в 0.55 говорит о слабой связи между параметрами. 

**Изучим зависимость цены продажи объекта - `last_price` от площади кухни - `kitchen_area`.**

In [ ]:
df['last_price'].corr(df['kitchen_area']).round(2)

In [ ]:
df.plot(x='kitchen_area', y='last_price', kind='scatter', grid=True, figsize=(10, 5))
plt.title('Зависимость цены объекта от площади кухни')
plt.xlabel('Площадь кухни')
plt.ylabel('Цена объекта')

Корреляция в 0.45 говорит о слабой связи между параметрами. 

**Построим тепловую карту зависимостей**

In [ ]:
df_corr = df[['city_centers_nearest'
              , 'last_price'
              , 'total_area'
              , 'living_area'
              , 'kitchen_area'
              , 'rooms'
              , 'ceiling_height'
              , 'floor'
              , 'day_of_the_week'
              , 'month'
              , 'year']]

plt.figure(figsize=(12, 10))
sns.heatmap(df_corr.corr(), xticklabels=df_corr.corr().columns,\
            yticklabels=df_corr.corr().columns, cmap='RdYlGn', center=0, annot=True)

Вывод

- Наибольшее влияние на цену продажи квартиры окзывает: общая и жилая площади. Удаленность от центра имеет обратную зависимость, чем она больше, тем цена ниже. Остальные показатели слабо коррелируют с ценой.

---

Посчитаем среднюю цену одного квадратного метра в 10 населённых пунктах с наибольшим числом объявлений. Выделим населённые пункты с самой высокой и низкой стоимостью квадратного метра.

In [ ]:
# Выведем таблицу с колонками count и mean 
localities_top_10 = df.pivot_table(index = 'locality_name'
                                   , values = 'price_square_meter'
                                   , aggfunc=['count', 'mean'])

# Присвоим пересменной две колонки
localities_top_10.columns = ['count', 'mean']

# Выполним сортировку по убыванию по столбцу count, положим результат в переменную
localities_top_10 = localities_top_10.sort_values('count', ascending = False).head(10)

# Выведем результат на экран
localities_top_10

In [ ]:
# Выведем среднюю объектов с самой высокой стоимостью
localities_top_10[localities_top_10['mean'] == localities_top_10['mean'].max()]

In [ ]:
# Выведем среднюю объектов с самой низкой стоимостью
localities_top_10[localities_top_10['mean'] == localities_top_10['mean'].min()]

---

Выделим квартиры в Санкт-Петербурге с помощью столбца locality_name и вычислите среднюю цену каждого километра. Опишем, как стоимость объектов зависит от расстояния до центра города.

In [ ]:
# Cоздадим новый объект с копией данных и индексов вызывающего объекта с
# использованием метода copy.
apartments_in_spb = df.query('locality_name == "Санкт-Петербург"').copy()
apartments_in_spb['city_centers_nearest'] = apartments_in_spb['city_centers_nearest'].astype(int)

city_center = apartments_in_spb.pivot_table(index='city_centers_nearest'
                                            , values='price_square_meter'
                                            , aggfunc='mean')
city_center.plot(figsize=(10, 5))
city_center

График резко меняется после 6 км, определим его как центр.

In [ ]:
apartments_in_spb_center = apartments_in_spb.query('city_centers_nearest <= 6')


In [ ]:
apartments_in_spb_center['total_area'].hist(bins=100,figsize=(10, 5))

plt.title("Распределение объектов по площади в центре")
plt.ylabel("Количество объектов")
plt.xlabel("Площадь объекта")

In [ ]:
apartments_in_spb_center['total_area'].hist(bins=100,figsize=(10,5))

df['total_area'].hist(bins=100,alpha=0.3,figsize=(10, 5))

plt.title("Распределение всех объектов по площади и объектов в центре")
plt.ylabel("Количество объектов")
plt.xlabel("Площадь объекта")
plt.xlim(0, 125)

In [ ]:
apartments_in_spb_center['total_area'].describe()

In [ ]:
df['total_area'].describe()

Медианное значение общей площади для центра увеличилось с 61 до 47 метров

In [ ]:
df.query('last_price < 12_000_000')['last_price'].hist(bins=100,figsize=(10, 5))

apartments_in_spb_center.query('last_price < 12_000_000')['last_price'].hist(bins=100, figsize=(10, 5))

plt.title("Распределение цен на объекты до 12 млн")
plt.ylabel("Количество объектов")
plt.xlabel("Цена 12 млн.")

In [ ]:
apartments_in_spb_center['last_price'].describe()

In [ ]:
df['last_price'].describe()

Цены на квартиры в центре выше чем в других районах. Видно, что медианное значение цены квартиры составляет 4,290,000.00, а в центре - 6,990,000.00, что превышает предыдущий показатель более чем в 1.5 раза.

---

### Общий вывод

В процессе выполнения проекта изучили данные о продаже квартир в Санкт-Петербурге и Ленинградской области. В таблице изначально было представлено 364 населенных пункта, часть относятся к районам Санкт-Петербурга часть к Ленинградской области. На Санкт-Петербург приходится наибольшее количество строк таблицы.

Значительные пропуски в данных относятся к описательной части объекта недвижимости:
- квартира или апартаменты - 88%;
- наличие балкона - 48%;
- высота потолка - 39%.

Так же значительная часть пропусков обнаружилась в значениях локации квартир: удаленность от центра , аэропорта или наличия рядом парков и водоёмов. Там где это было целесообразно, заменили пропуски на медианные значения. Значительная часть пропусков осталась не заполнена, но эти пропуски в основном относятся к объектам недвижимости Ленинградской области, где расстояние до центра и до аэропорта видимо не столь актуально.
Пропуски в колонке living_area были заменены пропорционально доле жилой площади в размере общей площади.

Также частично были изменены типы данных, метры заменены на километры с округлением до целого. Это поможет облегчить восприятие информации.

Значительно на разброс данных повлияли выбросы. При рассмотрении значения цены продажи, были обнаружены неординарные значения минимальной и максимальной цены. Минимальную цену исправли как ошибку заполнения, максимальную цены не трогали, видимо это реальная цена квартиры. Медиана цены продажи приходится на 4.26 млн, а среднее значение на 4.65 млн. Показатели сильно расходятся, что является результатом квартир с очень высокой стоимость, например максимальная цена  квартиры, может достигать 160 млн., а минимальная цена квартиры может составлять 0,43 млн.

Одними из важнейших параметров, из которых складывается стоимость квартиры - это площадь и месторасположение. Чем больше площадь и ближе к центру локация, тем дороже объект недвижимости.

При продаже квартиры стоит ориентироваться на точность заполнения данных и время подачи объявления, так как явно прослеживается сезонная активность покупателей. В среднем, при разумной оценке стоимости квартиры срок продаж должен составлять 2 - 3 месяца. 

**Чек-лист готовности проекта**

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл
- [x]  файлы изучены (выведены первые строки, метод `info()`, гистограммы и т.д.)
- [x]  определены пропущенные значения
- [x]  заполнены пропущенные значения там, где это возможно
- [x]  есть пояснение, какие пропущенные значения обнаружены
- [x]  изменены типы данных
- [x]  есть пояснение, в каких столбцах изменены типы и почему
- [x]  устранены неявные дубликаты в названиях населённых пунктов
- [x]  устранены редкие и выбивающиеся значения (аномалии) во всех столбцах
- [x]  посчитано и добавлено в таблицу: цена одного квадратного метра
- [x]  посчитано и добавлено в таблицу: день публикации объявления (0 - понедельник, 1 - вторник и т.д.)
- [x]  посчитано и добавлено в таблицу: месяц публикации объявления
- [x]  посчитано и добавлено в таблицу: год публикации объявления
- [x]  посчитано и добавлено в таблицу: тип этажа квартиры (значения — «первый», «последний», «другой»)
- [x]  посчитано и добавлено в таблицу: расстояние в км до центра города
- [x]  изучены и описаны следующие параметры:
        - общая площадь;
        - жилая площадь;
        - площадь кухни;
        - цена объекта;
        - количество комнат;
        - высота потолков;
        - этаж квартиры;
        - тип этажа квартиры («первый», «последний», «другой»);
        - общее количество этажей в доме;
        - расстояние до центра города в метрах;
        - расстояние до ближайшего аэропорта;
        - расстояние до ближайшего парка;
        - день и месяц публикации объявления
- [x]  построены гистограммы для каждого параметра
- [x]  выполнено задание: "Изучите, как быстро продавались квартиры (столбец days_exposition). Этот параметр показывает, сколько дней «висело» каждое объявление.
    - Постройте гистограмму.
    - Посчитайте среднее и медиану.
    - В ячейке типа markdown опишите, сколько обычно занимает продажа. Какие продажи можно считать быстрыми, а какие — необычно долгими?"
- [x]  выполнено задание: "Какие факторы больше всего влияют на общую (полную) стоимость объекта? Постройте графики, которые покажут зависимость цены от указанных ниже параметров. Для подготовки данных перед визуализацией вы можете использовать сводные таблицы."
        - общей площади;
        - жилой площади;
        - площади кухни;
        - количество комнат;
        - типа этажа, на котором расположена квартира (первый, последний, другой);
        - даты размещения (день недели, месяц, год);
- [x]  выполнено задание: "Посчитайте среднюю цену одного квадратного метра в 10 населённых пунктах с наибольшим числом объявлений. Выделите населённые пункты с самой высокой и низкой стоимостью квадратного метра. Эти данные можно найти по имени в столбце `locality_name`."
- [x]  выполнено задание: "Ранее вы посчитали расстояние до центра в километрах. Теперь выделите квартиры в Санкт-Петербурге с помощью столбца `locality_name` и вычислите среднюю цену каждого километра. Опишите, как стоимость объектов зависит от расстояния до центра города."
- [x]  в каждом этапе есть промежуточные выводы
- [x]  есть общий вывод